#### To save the state of the file automatically every 60 seconds

In [1]:
%autosave 60

Autosaving every 60 seconds


#### Importing all the required libraries

In [2]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

import statsmodels.api as sm
from statsmodels.tools.eval_measures import meanabs
import dask.dataframe as dd

#### To read data from the csv file and converting it to multiple parquet files in a directory. Need to run only once.

In [3]:
# brakes = dd.read_csv("/home/tai21/AAP/brakes/pi/brakes_pi.csv", sep='|',dtype={'ON_HAND': 'object'})
# brakes.to_parquet('/home/tai21/AAP/brakes/pi/parquet/', schema="infer")

#### Read parquet file

In [4]:
data = dd.read_parquet('/home/tai21/AAP/brakes/pi/parquet/part*')

#### Filtering out stores with store number 1020

In [5]:
#Dropping store number 1020, since it is for ecommerce
data= data[data['STORE_NUMBER']!=1020]
data

,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,PCT_BLUE_COLLAR,MEDIAN_HOUSEHOLD_INCOME,ESTABLISHMENTS,SKU_EXISTENCE_PY,SKU_EXISTENCE_CY,SKU_STORE_PDQ_PY,SKU_STORE_PDQ_CY,TOTAL_VIO_PY,TOTAL_VIO_CY,UNADJUSTED_TOTAL_VIO_PY,UNADJUSTED_TOTAL_VIO_CY,LOST_QTY_PY,LOST_QTY_CY,SS_SALES_PY,SS_SALES_CY,AVG_CLUSTER_UNIT_SALES_PY,AVG_CLUSTER_UNIT_SALES_CY,AVG_CLUSTER_LOST_SALES_PY,AVG_CLUSTER_LOST_SALES_CY,ROAD_QUALITY_INDEX,VIO_COMPARED_TO_CLUSTER_PY,VIO_COMPARED_TO_CLUSTER_CY,ADJUSTED_AVG_CLUSTER_SALES_PY,ADJUSTED_AVG_CLUSTER_SALES_CY,QTY_SOLD_PPY,QTY_SOLD_PY,QTY_SOLD_CY,AVG_CLUSTER_TOTAL_SALES_PY,AVG_CLUSTER_TOTAL_SALES_CY,SALES_SIGNAL_PY,SALES_SIGNAL_CY,FAILURE_SALES_PY,FAILURE_SALES_CY,LIFECYCLE_PY,LIFECYCLE_CY,ADJUSTED_LIFECYCLE_PY,ADJUSTED_LIFECYCLE_CY,PCT_OF_LIFECYCLE_REMAINING,LIFECYCLE_PRE_PEAK_POST,ADJ_AVG_CLUSTER_LOST_SALES_PY,ADJ_AVG_CLUSTER_LOST_SALES_CY,ADJ_AVG_CLUSTER_TOTAL_SALES_PY,ADJ_AVG_CLUSTER_TOTAL_SALES_CY,SOLD_SINCE_MAXI,cy_unit_sales,py_unit_sales,ppy_unit_sales,cy_gross_sales,py_gross_sales,ppy_gross_sales,cy_sales_cost,py_sales_cost,ppy_sales_cost,other_unit_sales,other_gross_sales,other_sales_cost,PROJECTED_GROWTH_PCT_PY,PROJECTED_GROWTH_PCT_CY,OTHER_UNIT_PLS_LOST_SALES_PPY,OTHER_UNIT_PLS_LOST_SALES_PY,OTHER_UNIT_PLS_LOST_SALES_CY,LOOKUP_CNT_PY,LOOKUP_CNT_CY,WEIGHTED_LOOKUP_CNT_PY,WEIGHTED_LOOKUP_CNT_CY,CY_PERIODS_IN_STOCK,WAIT_TIME_BUCKET,MPOG_ID,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
npartitions=251,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,object,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,object,float64,float64,float64,float64,float64,float64,object,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [6]:
data.head()

,SKU_NUMBER,STORE_NUMBER,PART_TYPE,PLATFORM_CLUSTER_NAME,APPLICATION_COUNT,POP_EST_CY,POP_DENSITY_CY,PCT_WHITE,AGE,PCT_COLLEGE,...,DATA_CLASSIFICATION,cy_qty_sold_transfer,cy_qty_sold_on_hand,py_qty_sold_transfer,py_qty_sold_on_hand,ppy_qty_sold_transfer,ppy_qty_sold_on_hand,ON_HAND,fy_ts_forecast,cy_ts_forecast
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,10487658,1222,None,69.0,0,8090.64,0.001195,0.06,36.63,0.13,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
1,12193934,5047,DISC BRAKE PAD SET,NaN,0,0.00,0.000000,0.50,40.00,0.25,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
2,10487658,2120,None,30.0,0,11121.04,0.007502,0.26,35.59,0.14,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
3,12193934,5150,DISC BRAKE PAD SET,30.0,0,15388.90,0.004061,0.29,33.91,0.10,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
4,10487658,1261,None,77.0,0,13504.18,0.008951,0.02,36.68,0.11,...,NON PR NO EPC,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN


#### Dropping all the columns containing data from the current year except QTY_SOLD_CY and PROJECT_GROWTH_PCT_CY

In [7]:
from tqdm import tqdm
drop_columns = []
for i in tqdm(data.columns):
    if ('_cy' in i.lower() or 'cy_' in i.lower()) and i!='QTY_SOLD_CY' and i!='PROJECTED_GROWTH_PCT_CY':
        drop_columns.append(i)
    else:
        continue

100%|██████████| 88/88 [00:00<00:00, 885128.90it/s]


In [8]:
data.columns

Index(['SKU_NUMBER', 'STORE_NUMBER', 'PART_TYPE', 'PLATFORM_CLUSTER_NAME',
       'APPLICATION_COUNT', 'POP_EST_CY', 'POP_DENSITY_CY', 'PCT_WHITE', 'AGE',
       'PCT_COLLEGE', 'PCT_BLUE_COLLAR', 'MEDIAN_HOUSEHOLD_INCOME',
       'ESTABLISHMENTS', 'SKU_EXISTENCE_PY', 'SKU_EXISTENCE_CY',
       'SKU_STORE_PDQ_PY', 'SKU_STORE_PDQ_CY', 'TOTAL_VIO_PY', 'TOTAL_VIO_CY',
       'UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 'LOST_QTY_PY',
       'LOST_QTY_CY', 'SS_SALES_PY', 'SS_SALES_CY',
       'AVG_CLUSTER_UNIT_SALES_PY', 'AVG_CLUSTER_UNIT_SALES_CY',
       'AVG_CLUSTER_LOST_SALES_PY', 'AVG_CLUSTER_LOST_SALES_CY',
       'ROAD_QUALITY_INDEX', 'VIO_COMPARED_TO_CLUSTER_PY',
       'VIO_COMPARED_TO_CLUSTER_CY', 'ADJUSTED_AVG_CLUSTER_SALES_PY',
       'ADJUSTED_AVG_CLUSTER_SALES_CY', 'QTY_SOLD_PPY', 'QTY_SOLD_PY',
       'QTY_SOLD_CY', 'AVG_CLUSTER_TOTAL_SALES_PY',
       'AVG_CLUSTER_TOTAL_SALES_CY', 'SALES_SIGNAL_PY', 'SALES_SIGNAL_CY',
       'FAILURE_SALES_PY', 'FAILURE_SALES_CY', '

#### Dropping all the redundant, categorical and unnecessary columns

In [9]:
drop_columns.extend(['PLATFORM_CLUSTER_NAME','UNADJUSTED_TOTAL_VIO_PY', 'UNADJUSTED_TOTAL_VIO_CY', 
                     'SS_SALES_PY', 'FAILURE_SALES_PY','LIFECYCLE_PY', 'AVG_CLUSTER_LOST_SALES_PY', 
                     'other_unit_sales', 'other_gross_sales', 'AVG_CLUSTER_TOTAL_SALES_PY', 'other_sales_cost',
                     'LOOKUP_CNT_PY', 'WEIGHTED_LOOKUP_CNT_PY', 'py_qty_sold_transfer', 'py_qty_sold_on_hand',
                     'ppy_qty_sold_transfer', 'ppy_qty_sold_on_hand', 'ON_HAND', 'fy_ts_forecast','MPOG_ID', 
                     'DATA_CLASSIFICATION', 'PART_TYPE', 'WAIT_TIME_BUCKET','py_unit_sales','ppy_unit_sales',
                     'ppy_gross_sales', 'py_sales_cost','ppy_sales_cost','ROAD_QUALITY_INDEX','TOTAL_VIO_PY','ESTABLISHMENTS'])
data = data.drop(drop_columns,axis=1)
data.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 29 entries, SKU_NUMBER to OTHER_UNIT_PLS_LOST_SALES_PY
dtypes: float64(21), int64(8)

#### Imputation

In [10]:
data.OTHER_UNIT_PLS_LOST_SALES_PY = data.OTHER_UNIT_PLS_LOST_SALES_PY.fillna(0)
data.OTHER_UNIT_PLS_LOST_SALES_PPY = data.OTHER_UNIT_PLS_LOST_SALES_PPY.fillna(0)
data.PROJECTED_GROWTH_PCT_CY = data.PROJECTED_GROWTH_PCT_CY.fillna('mean')
data.PROJECTED_GROWTH_PCT_PY = data.PROJECTED_GROWTH_PCT_PY.fillna('mean')

#### Deleting all the SKUs with zero gross sales in the previous year and imputing negative values of QTY_SOLD_CY to zero

In [11]:
sku_list = set(data[data['py_gross_sales']!=0]['SKU_NUMBER'])
data = data[data['SKU_NUMBER'].isin(sku_list)]
data[data['QTY_SOLD_CY']<0]['QTY_SOLD_CY'] = 0

In [12]:
data.PROJECTED_GROWTH_PCT_PY = data.PROJECTED_GROWTH_PCT_PY.astype(float)
data.PROJECTED_GROWTH_PCT_CY = data.PROJECTED_GROWTH_PCT_CY.astype(float)

#### Defining predictor and target variables

In [13]:
y = data['QTY_SOLD_CY'].copy()
x = data.drop(['QTY_SOLD_CY','SKU_NUMBER','STORE_NUMBER'],axis=1).copy()

#### Converting the Dask dataframes back to Pandas dataframes

In [14]:
x = x.compute()
y = y.compute()
print(type(x))
print(type(y))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [15]:
# Split categorical and numerical Columns
num_cols = []
cat_cols = []
for i in x.columns:
    if x[i].dtype=='object':
        cat_cols.append(i)
    else:
        num_cols.append(i)

In [16]:
x.columns

Index(['APPLICATION_COUNT', 'PCT_WHITE', 'AGE', 'PCT_COLLEGE',
       'PCT_BLUE_COLLAR', 'MEDIAN_HOUSEHOLD_INCOME', 'SKU_EXISTENCE_PY',
       'SKU_STORE_PDQ_PY', 'LOST_QTY_PY', 'AVG_CLUSTER_UNIT_SALES_PY',
       'VIO_COMPARED_TO_CLUSTER_PY', 'ADJUSTED_AVG_CLUSTER_SALES_PY',
       'QTY_SOLD_PPY', 'QTY_SOLD_PY', 'SALES_SIGNAL_PY',
       'ADJUSTED_LIFECYCLE_PY', 'PCT_OF_LIFECYCLE_REMAINING',
       'LIFECYCLE_PRE_PEAK_POST', 'ADJ_AVG_CLUSTER_LOST_SALES_PY',
       'ADJ_AVG_CLUSTER_TOTAL_SALES_PY', 'SOLD_SINCE_MAXI', 'py_gross_sales',
       'PROJECTED_GROWTH_PCT_PY', 'PROJECTED_GROWTH_PCT_CY',
       'OTHER_UNIT_PLS_LOST_SALES_PPY', 'OTHER_UNIT_PLS_LOST_SALES_PY'],
      dtype='object')

#### Standardization

In [17]:
all_cols = x.columns
ct = ColumnTransformer([
        ('scaler', StandardScaler(), all_cols)
    ], remainder='passthrough')

x = pd.DataFrame(ct.fit_transform(x), columns=all_cols)

In [18]:
ex_cols=['QTY_SOLD_PY', 'QTY_SOLD_PPY']
t_cols = x.columns
for col1 in ex_cols:
    for col2 in t_cols:
        x[col1 + '&' + col2] = x[col1]*x[col2]

#### Splitting the data into random training and test subsets

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [20]:
y_train = y_train.tolist()
y_test = y_test.tolist()

#### Performing Ordinary Least Squares on the training data using statsmodels

In [21]:
%%time
#x = sm.add_constant(x)
olsmodel = sm.OLS(y_train,x_train)
results = olsmodel.fit()
results.summary()

CPU times: user 8min 54s, sys: 2min 43s, total: 11min 37s
Wall time: 25.6 s


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.834
Model:                            OLS   Adj. R-squared (uncentered):              0.834
Method:                 Least Squares   F-statistic:                          5.282e+05
Date:                Thu, 07 Apr 2022   Prob (F-statistic):                        0.00
Time:                        10:42:04   Log-Likelihood:                     -9.0498e+06
No. Observations:             7468464   AIC:                                  1.810e+07
Df Residuals:                 7468393   BIC:                                  1.810e+07
Df Model:                          71                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
APPLICATION_COUNT                              -0.0017      0.000     -5.360      0.000      -0.002      -0.001
PCT_WHITE                                       0.0047      0.000     12.670      0.000       0.004       0.005
AGE                                            -0.0077      0.000    -18.356      0.000      -0.009      -0.007
PCT_COLLEGE                                     0.0030      0.001      5.839      0.000       0.002       0.004
PCT_BLUE_COLLAR                                 0.0067      0.001     10.789      0.000       0.005       0.008
MEDIAN_HOUSEHOLD_INCOME                        -0.0116      0.000    -31.081      0.000      -0.012      -0.011
SKU_EXISTENCE_PY                               -0.4272      0.001   -292.783      0.000      -0.430      -0.424
SKU_STORE_PDQ_PY                                0.5626      0.001    375.339      0.000       0.560       0.566
LOST_QTY_PY                                    -0.0002      0.000     -0.561      0.575      -0.001       0.000
AVG_CLUSTER_UNIT_SALES_PY                       0.0667      0.002     41.406      0.000       0.064       0.070
VIO_COMPARED_TO_CLUSTER_PY                      0.0024      0.000      7.437      0.000       0.002       0.003
ADJUSTED_AVG_CLUSTER_SALES_PY                -3.15e+04   1.44e+05     -0.218      0.827   -3.14e+05    2.51e+05
QTY_SOLD_PPY                                   -0.3135      0.003   -112.732      0.000      -0.319      -0.308
QTY_SOLD_PY                                    -0.1669      0.087     -1.925      0.054      -0.337       0.003
SALES_SIGNAL_PY                                -0.0048      0.087     -0.055      0.956      -0.175       0.165
ADJUSTED_LIFECYCLE_PY                          -0.0129      0.000    -26.600      0.000      -0.014      -0.012
PCT_OF_LIFECYCLE_REMAINING                      0.0092      0.001     17.948      0.000       0.008       0.010
LIFECYCLE_PRE_PEAK_POST                        -0.0018      0.000     -3.943      0.000      -0.003      -0.001
ADJ_AVG_CLUSTER_LOST_SALES_PY                2.348e+04   1.07e+05      0.219      0.827   -1.87e+05    2.34e+05
ADJ_AVG_CLUSTER_TOTAL_SALES_PY                3.15e+04   1.44e+05      0.218      0.827   -2.51e+05    3.14e+05
SOLD_SINCE_MAXI                                 0.3824      0.000    972.649      0.000       0.382       0.383
py_gross_sales                                 -0.1446      0.002    -85.996      0.000      -0.148      -0.141
PROJECTED_GROWTH_PCT_PY                         0.1484      0.000    310.796      0.000       0.147       0.149
PROJECTED_GROWTH_PCT_CY                         0.0257      0.000     69

#### Removing all the insignificant variables until only significant variables are left

In [22]:
count=0

while (results.pvalues[results.pvalues > 0.05].index.tolist()):
    print("Iteration: ", count)
    lst = list(results.pvalues[results.pvalues > 0.05].index)
    x.drop(lst, axis='columns', inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
    y_train = y_train.tolist()
    y_test = y_test.tolist()
    olsmodel = sm.OLS(y_train,x_train)
    results = olsmodel.fit()
    
    count += 1

Iteration:  0
Iteration:  1


In [23]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.834
Model:                            OLS   Adj. R-squared (uncentered):              0.834
Method:                 Least Squares   F-statistic:                          6.239e+05
Date:                Thu, 07 Apr 2022   Prob (F-statistic):                        0.00
Time:                        10:44:10   Log-Likelihood:                     -9.0557e+06
No. Observations:             7468464   AIC:                                  1.811e+07
Df Residuals:                 7468404   BIC:                                  1.811e+07
Df Model:                          60                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
APPLICATION_COUNT                             -0.0022      0.000     -7.207      0.000      -0.003      -0.002
PCT_WHITE                                      0.0046      0.000     12.379      0.000       0.004       0.005
AGE                                           -0.0077      0.000    -18.381      0.000      -0.009      -0.007
PCT_COLLEGE                                    0.0030      0.001      5.985      0.000       0.002       0.004
PCT_BLUE_COLLAR                                0.0066      0.001     10.604      0.000       0.005       0.008
MEDIAN_HOUSEHOLD_INCOME                       -0.0114      0.000    -30.512      0.000      -0.012      -0.011
SKU_EXISTENCE_PY                              -0.4261      0.001   -292.100      0.000      -0.429      -0.423
SKU_STORE_PDQ_PY                               0.5576      0.001    372.205      0.000       0.555       0.560
AVG_CLUSTER_UNIT_SALES_PY                      0.1471      0.001    205.194      0.000       0.146       0.148
VIO_COMPARED_TO_CLUSTER_PY                     0.0088      0.000     29.403      0.000       0.008       0.009
QTY_SOLD_PPY                                  -0.4229      0.002   -233.814      0.000      -0.426      -0.419
ADJUSTED_LIFECYCLE_PY                         -0.0134      0.000    -27.537      0.000      -0.014      -0.012
PCT_OF_LIFECYCLE_REMAINING                     0.0094      0.001     18.221      0.000       0.008       0.010
LIFECYCLE_PRE_PEAK_POST                       -0.0019      0.000     -4.201      0.000      -0.003      -0.001
SOLD_SINCE_MAXI                                0.3810      0.000    995.202      0.000       0.380       0.382
py_gross_sales                                -0.1792      0.001   -124.523      0.000      -0.182      -0.176
PROJECTED_GROWTH_PCT_PY                        0.1476      0.000    309.471      0.000       0.147       0.149
PROJECTED_GROWTH_PCT_CY                        0.0263      0.000     71.129      0.000       0.026       0.027
OTHER_UNIT_PLS_LOST_SALES_PPY                 -0.3851      0.002   -179.737      0.000      -0.389      -0.381
OTHER_UNIT_PLS_LOST_SALES_PY                   0.4385      0.002    195.838      0.000       0.434       0.443
QTY_SOLD_PY&APPLICATION_COUNT                  0.0328      0.000    117.263      0.000       0.032       0.033
QTY_SOLD_PY&PCT_WHITE                         -0.0179      0.001    -25.509      0.000      -0.019      -0.017
QTY_SOLD_PY&AGE                               -0.0370      0.001    -44.227      0.000      -0.039      -0.035
QTY_SOLD_PY&PCT_COLLEGE                        0.0117      0.001     11.384      0.000       0.010

#### Evaluating OLS model on the test set

In [24]:
y_preds = results.predict(x_test)
residuals = y_test - y_preds

MSE = np.sum(residuals**2)/len(y_test)
print("Mean Squared Error: ", MSE)

MAE = meanabs(y_test, y_preds)
print("Mean Absolute Error: ", MAE)

Mean Squared Error:  0.6312003689992379
Mean Absolute Error:  0.18694214906312662


#### Elastic Net Regression

In [26]:
%%time

en = ElasticNet()

print("Splitting done, proceeding with EN")

model = en.fit(x_train,y_train)
y_pred = en.predict(x_test)

print('R-squared:', r2_score(y_test,y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

# # #print(stime - time.time())

Splitting done, proceeding with EN
R-squared: 0.7473133041952764
Mean Squared Error: 0.9576945110262576
Mean Absolute Error:  0.36475849900611296
CPU times: user 12min 54s, sys: 8.71 s, total: 13min 3s
Wall time: 15.5 s


#### Lasso Regression

In [27]:
%%time

lasso = Lasso()

model = lasso.fit(x_train,y_train)
y_pred = lasso.predict(x_test)

print("R-squared: ", r2_score(y_test,y_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

R-squared:  0.7042987322782275
Mean Squared Error:  1.1207217700907297
Mean Absolute Error:  0.4080716530346923
CPU times: user 14min, sys: 9.43 s, total: 14min 10s
Wall time: 16.4 s


#### Ridge Regression

In [29]:
%%time

ridge = Ridge()

model = ridge.fit(x_train,y_train)
y_pred = ridge.predict(x_test)

print("R-squared: ", r2_score(y_test,y_pred))
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
print("Mean Absolute Error: ", meanabs(y_test, y_pred))

R-squared:  0.8343423710249109
Mean Squared Error:  0.6278502375197131
Mean Absolute Error:  0.176405810810434
CPU times: user 16.1 s, sys: 651 ms, total: 16.7 s
Wall time: 4.16 s
